In [43]:

#Things That can be neeed to change for trainig purpose ?
# Collate Function

import os
import pickle
import yaml
import argparse

import torch
from torch.utils import data
import numpy as np
from scipy.spatial import Delaunay
from tqdm import tqdm

from model.build_model import build_gcn, build_netvlad, build_seqnet, build_airobj

from datasets.mp3d_airloc.mp3d import mp3d
from datasets.utils.batch_collator import eval_custom_collate
from model.graph_models.gcn import GCN
from model.graph_models.gat import  GAT

dataset = mp3d(base_dir="/home/aryan/Mp3d_dataset/x_view",datasets= ["mp3d"])

loader = data.DataLoader(dataset=dataset, batch_size=1, collate_fn=eval_custom_collate,shuffle=False)
batch = next(iter(loader))

Dataset Name =  mp3d
Scene Name =  qoiz87JEwZ2
Room Name =  room3
Room Name =  room4
Room Name =  lobby2
Room Name =  room1
Room Name =  lobby1
Room Name =  room2
Room Name =  kitchen
Room Name =  bathroom1
Room Name =  dining_room


In [44]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
ann_masks, points, roomnames = batch[0], batch[1], batch[2]

batch_objects = []
for i in range(len(ann_masks)): # Batch Size
    keypoints = points[i]['points']
    descriptors = points[i]['point_descs']

    
    image_objects = {}
    image_objects['points'] = []
    image_objects['descs'] = []
    image_objects['ids'] = [] 

    for a in range(len(ann_masks[i])):
        ann_mask = ann_masks[i][a]['mask']
        object_filter = ann_mask[keypoints[:,0].T,keypoints[:,1].T]

        np_obj_pts = keypoints[np.where(object_filter==1)[0]].numpy()

        obj_id = str(ann_masks[i][a]['id']) 
        
        image_objects['points'].append(keypoints[np.where(object_filter==1)[0]].float().to(device))
        image_objects['descs'].append(descriptors[np.where(object_filter==1)[0]].float().to(device))
        image_objects['ids'].append(obj_id)
    
    batch_objects.append(image_objects)

In [45]:
configs = "config/eval_Airloc.yaml"
f = open(configs, 'r', encoding='utf-8')
configs = f.read()
configs = yaml.safe_load(configs)
configs['use_gpu'] = 1
configs['num_gpu'] = [0]
configs['public_model'] = 0

netvlad_model = build_netvlad(configs)
netvlad_model.eval()

Finish cuda loading
load model from ../models/netvlad_model.pth


DataParallel(
  (module): NetVLADDescriptor(
    (netvlad): NetVLAD(
      (conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)

In [46]:
batch_decs = []
for image in batch_objects:
    new_image_desc = []
    for i in image['descs']:
        if i.shape[0]>5:
            new_image_desc.append(i)
    batch_decs.append(new_image_desc)

In [47]:
obj_descs = netvlad_model(batch_decs[0])

In [48]:
nodes = torch.tensor(obj_descs.shape[0])
adj = torch.ones(nodes,nodes)

GNN

In [49]:
model = GCN(nfeat=8192, nhid=4096, nclass=4096, dropout=0.5).to(device)


In [53]:
model.train()
output = model(obj_descs.to(device), adj.to(device))

In [54]:
output.shape

torch.Size([4096])

## Test SuperPoint Object Decriptor


In [ ]:
import os
import cv2
import copy
import torch
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt

# Image taken after looking at the imags from whivh the points are  taken.
img_path = '/home/aryan/Mp3d_dataset/x_view/mp3d/qoiz87JEwZ2/rooms/room3/raw_data/0_rgb.png'

src = [cv2.imread(img_path)]

In [ ]:
batch_pts = []
batch_ids = []
for image in batch_objects:
    batch_pts.append(image['points'])
    batch_ids.append(image['ids'])
    
images_pts  = batch_pts[0]

In [ ]:
def overlay_points(images, points_output, color=(255,0,0)):
  new_images = []
  for image, points_o in zip(images, points_output):
    image1 = image.copy()
    points = points_o.cpu().numpy()
    if len(points) == 0:
      print("no points")
    for i in range(len(points)):
      x = points[i][1].astype(int)
      y = points[i][0].astype(int)
      if x < 0:
        continue
      cv2.circle(image1, (x,y), 10, color, thickness=-1)
    new_images.append(image1)
  return new_images

In [ ]:
images = []
for i, pt in enumerate(images_pts):
    img = overlay_points(src,[pt])
    print("ID = ",batch_ids[0][i] )
    fig = plt.figure(i)
    plt.imshow(img[0])
    plt.show()
    images.append(img)

## Triplet_loss


In [8]:
import os
import pickle
import yaml
import argparse

import torch
from torch.utils import data
import numpy as np
from scipy.spatial import Delaunay
from tqdm import tqdm

from model.build_model import build_gcn, build_netvlad, build_seqnet, build_airobj
from model.graph_models.gcn import GCN
from datasets.mp3d_airloc.mp3d_triplet import mp3d
from datasets.utils.batch_collator import eval_custom_collate

dataset = mp3d(base_dir="/home/aryan/Mp3d_dataset/x_view",datasets= ["mp3d"])

loader = data.DataLoader(dataset=dataset, batch_size=4, collate_fn=eval_custom_collate,shuffle=False)

configs = "config/eval_Airloc.yaml"
f = open(configs, 'r', encoding='utf-8')
configs = f.read()
configs = yaml.safe_load(configs)
configs['use_gpu'] = 1
configs['num_gpu'] = [0]
configs['public_model'] = 0

netvlad_model = build_netvlad(configs)
netvlad_model.eval()

Dataset Name =  mp3d
Scene Name =  qoiz87JEwZ2
Room Name =  room3
Room Name =  room4
Room Name =  lobby2
Room Name =  room1
Room Name =  lobby1
Room Name =  room2
Room Name =  kitchen
Room Name =  bathroom1
Room Name =  dining_room
Finish cuda loading
load model from ../models/netvlad_model.pth


DataParallel(
  (module): NetVLADDescriptor(
    (netvlad): NetVLAD(
      (conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)

In [9]:
triplet_loss = torch.nn.TripletMarginLoss(margin=1.0, p=2)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = GCN(nfeat=8192, nhid=4096, nclass=4096, dropout=0.5).to(device)

for step, (anchor_img, positive_img, negative_img) in enumerate(tqdm(loader)):
    
    anchor_pts = images_to_point_decs(anchor_img)
    positive_pts = images_to_point_decs(positive_img)
    negative_pts = images_to_point_decs(negative_img)
    
    anchor_objs = points_to_obj_desc(anchor_pts)
    positive_objs = points_to_obj_desc(positive_pts)
    negative_objs = points_to_obj_desc(negative_pts)
    
    anchor_room = batch_gnn_processing(anchor_objs)
    positive_room = batch_gnn_processing(positive_objs)
    negative_room = batch_gnn_processing(negative_objs)
    
    output = triplet_loss(anchor_room, positive_room, negative_room)
    print(output)

  2%|▏         | 1/64 [00:02<02:53,  2.75s/it]

tensor(3.6518, device='cuda:0', grad_fn=<MeanBackward0>)


  3%|▎         | 2/64 [00:04<02:23,  2.31s/it]

tensor(10.1414, device='cuda:0', grad_fn=<MeanBackward0>)


  5%|▍         | 3/64 [00:06<02:09,  2.12s/it]

...

tensor(34.9488, device='cuda:0', grad_fn=<MeanBackward0>)


 41%|████      | 26/64 [01:04<01:43,  2.71s/it]

tensor(10.3752, device='cuda:0', grad_fn=<MeanBackward0>)


 41%|████      | 26/64 [01:05<01:35,  2.50s/it]


KeyboardInterrupt: 

In [3]:
def images_to_point_decs(batch_image):
    """This function is specific for the case where batch of imgaes is iterable in form og single image"""
    
    batch_objects = []
    
    for image in batch_image:  #batch
        if len(image)!=3:
            #The case in which rgb and/or depth data in input along with this 
            print("images_to_obj_decs not implement error !!")

        ann_masks, points, roomname = image[0], image[1], image[2]
        
        keypoints = points['points']
        descriptors = points['point_descs']

        
        image_objects = {}
        image_objects['points'] = []
        image_objects['descs'] = []
        image_objects['ids'] = [] 
        image_objects['room_image_name'] = roomname

        for a in range(len(ann_masks)):
            ann_mask = ann_masks[a]['mask']
            object_filter = ann_mask[keypoints[:,0].T,keypoints[:,1].T]

            np_obj_pts = keypoints[np.where(object_filter==1)[0]].numpy()

            obj_id = str(ann_masks[a]['id']) 
            
            image_objects['points'].append(keypoints[np.where(object_filter==1)[0]].float().to(device))
            image_objects['descs'].append(descriptors[np.where(object_filter==1)[0]].float().to(device))
            image_objects['ids'].append(obj_id) 
        
        batch_objects.append(image_objects)
    return batch_objects
    

In [4]:
def points_to_obj_desc(batch_objects):
    batch_decs = []
    for image_objects in batch_objects:
        #Takes only those objects whone no. points are more then rejection threshold
        batch_points = []
        for object_points in image_objects['descs']:
            if object_points.shape[0]>5:
                batch_points.append(object_points)
                
        object_desc = netvlad_model(batch_points)
        batch_decs.append(object_desc)
    return batch_decs

In [5]:
def batch_gnn_processing(batch_obj):
    model.train()
    batch_features = []
    #This is a temporary function, need to replace this with GCN descriptor similar to NetVlad Descriptor for batched processing.
    for obj in batch_obj:
        nodes = torch.tensor(obj.shape[0])
        adj = torch.ones(nodes,nodes)
        output= model(obj.to(device), adj.to(device))
        batch_features.append(output.squeeze())
    
    return torch.stack(batch_features)

In [2]:
import torch

In [36]:
a = torch.rand(4,30)
b = torch.rand(4,30)
c = torch.rand(4,30)

In [40]:
def accuracy(a,p,n):
    dist1 = (a - p).pow(2).sum(1)
    dist2 = (a - n).pow(2).sum(1)
    print(dist1,dist2)
    pred = (dist1 - dist2 ).cpu()
    return (pred > 0).sum()*1.0/dist1.size()[0]

In [41]:
a =accuracy(a,b,c)

tensor([3.7875, 3.9097, 4.5436, 6.3357]) tensor([3.8029, 4.2859, 4.4021, 4.1218])


In [39]:
a

tensor(0.2500)